In [194]:
import pandas as pd
import numpy as np

### Goal: to retrieve professionally collected weather data from an online source to predict coffee harvests in Minas Gerais, Brazil

##### This notebook will be used to resample, collect, and save median values of each selected measureable for different time periods to allow us to perform exploratory data analysis on trends and correlations between months and years.

In [224]:
#Files output by weather_api.py, and removing nulls
weather = pd.read_csv("/Users/sa12/Documents/Repositories/WeatherDataMG/weather_api/data/csv/measurements.csv").dropna()
harvest = pd.read_csv("/Users/sa12/Documents/Repositories/WeatherDataMG/weather_api/data/csv/br_final.csv").dropna()
extra = pd.read_csv("/Users/sa12/Documents/Repositories/WeatherDataMG/weather_api/data/csv/extra_measure.csv").dropna()

In [225]:
#set DatetimeIndex for easy resampling
weather1 = weather.set_index(pd.DatetimeIndex(weather["time"]))
extra1 = extra.set_index(pd.DatetimeIndex(weather["time"]))


In [226]:
monthly_temps = weather1.resample("ME")["temperature"].median()
#resampling tempurature_2m
yearly_temps = weather1.resample("YE")["temperature"].median()
extra_temps = extra1.resample("YE")["temperature"].median()


In [227]:
monthly_humidity = weather1.resample("ME")["humidity"].median()
#resampling realative_humidity
yearly_humidity = weather1.resample("YE")["humidity"].median()
extra_humidity = extra1.resample("YE")["humidity"].median()

In [228]:
monthly_rain = weather1.resample("ME")["precipitation"].median()
#resampling precipitation
yearly_rain = weather1.resample("YE")["precipitation"].median()
extra_rain = extra1.resample("YE")["precipitation"].median()

In [230]:
monthly_psr = weather1.resample("ME")["pressure"].median()
#resampling surface_pressure_2m
yearly_psr = weather1.resample("YE")["pressure"].median()
extra_psr = extra1.resample("YE")["pressure"].median()

##### This block creates dataframes of the combined resamples for each timeframe, than saves those to csv files at the weather_api/data/csv path

In [232]:
#saving the median values of each month
Monthly_Medians = pd.DataFrame({"temperature":monthly_temps, 
                                "humidity":monthly_humidity, 
                                "precipitation":monthly_rain, 
                                "pressure":monthly_psr
                                
                                })
Monthly_Medians.to_csv("/Users/sa12/Documents/Repositories/WeatherDataMG/weather_api/data/csv/MonthlyMedians.csv", index=True)
#saving the median values of each year
Yearly_Medians = pd.DataFrame({"temperature":yearly_temps, 
                               "humidity":yearly_humidity, 
                               "precipitation":yearly_rain, 
                               "pressure":yearly_psr})
Yearly_Medians.to_csv("/Users/sa12/Documents/Repositories/WeatherDataMG/weather_api/data/csv/YearlyMedians.csv", index=True)
Extra_Medians = pd.DataFrame({"temperature":extra_temps, 
                               "humidity":extra_humidity, 
                               "precipitation":extra_rain, 
                               "pressure":extra_psr})

##### Creating a single file with weather data in Minas Gerais taken from our web API combined with harvest data given to us in "br_final.csv"

In [223]:
minas_harvest = harvest[harvest["subdivision"] == "Minas Gerais"].drop_duplicates()
#a nifty iteration to take only the data from Minas Gerais. 
#If the bool is True, the whole rows data is collected
#setting index to year is important, we will merge based on year
minas_harvest = minas_harvest.set_index(minas_harvest["year"])
#Yearly_Medians already is resampled and index to Datetime
Yearly_Medians1 = Yearly_Medians.reset_index(drop=False)
#we remove that index and rename "time to "year" creating uniformity
Yearly_Medians1=Yearly_Medians1.rename(columns = {'time':'year'})
#create a new variable to hold only the datetime value of the "year" column
Yearly_Medians2 = pd.to_datetime(Yearly_Medians1["year"], format="mixed", yearfirst=True)
#take only the year from this and overwrite the year data in our original variable
Yearly_Medians1["year"] = Yearly_Medians2.dt.year
#and set the index to this single year, matching the format of minas_harvest
Yearly_Medians1.set_index(Yearly_Medians1["year"], inplace=True)
#Now we merge outer, combining the indcies and data correctly
harvest_weather = pd.merge(minas_harvest, Yearly_Medians1, how="outer", left_index=True, right_index=True)
#save to path weather_api/data/csv/harvest_weather.csv
harvest_weather.to_csv("/Users/sa12/Documents/Repositories/WeatherDataMG/weather_api/data/csv/harvest_weather.csv", index=True)

In [233]:
Extra_Medians1 = Extra_Medians.reset_index(drop=False)
#we remove that index and rename "time to "year" creating uniformity
Extra_Medians1=Extra_Medians1.rename(columns = {'time':'year'})
#create a new variable to hold only the datetime value of the "year" column
Extra_Medians2 = pd.to_datetime(Extra_Medians1["year"], format="mixed", yearfirst=True)
#take only the year from this and overwrite the year data in our original variable
Extra_Medians1["year"] = Extra_Medians2.dt.year
#and set the index to this single year, matching the format of minas_harvest
Extra_Medians1.set_index(Extra_Medians1["year"], inplace=True)
#Now we merge outer, combining the indcies and data correctly
extra_harvest_weather = pd.merge(minas_harvest, Extra_Medians1, how="outer", left_index=True, right_index=True)
extra_harvest_weather.tail()

,country,subdivision,type,million_60kgs_bag,year_x,nonbear_mill_trees,bear_mill_trees,nonbear_thous_hect,bear_thous_hect,trees_hect_bear,trees_hect_nonbear,avg_unemp_perc,year_y,temperature,humidity,precipitation,pressure
year,,,,,,,,,,,,,,,,,
2019,Brazil,Minas Gerais,Arabica,34.2,2019.0,1150.0,5740.0,335.0,2060.0,2786.0,3433.0,10.10,NaN,NaN,NaN,NaN,NaN
2020,Brazil,Minas Gerais,Arabica,29.1,2020.0,1230.0,5700.0,350.0,2040.0,2794.0,3514.0,12.75,NaN,NaN,NaN,NaN,NaN
2021,Brazil,Minas Gerais,Arabica,34.8,2021.0,1050.0,6200.0,320.0,2100.0,2952.0,3281.0,11.65,NaN,NaN,NaN,NaN,NaN
2022,Brazil,Minas Gerais,Arabica,24.5,2022.0,1500.0,6010.0,470.0,2010.0,2990.0,3191.0,7.15,2022.0,18.6,78.0,0.0,911.4
2023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023.0,20.7,78.0,0.0,913.4
